In [1]:
# !pip3 install -r requirements.txt

In [ ]:
import importlib
from lion_weapon import LionWeapon
from importer import Importer
from file_utils import get_filepaths_list
import lion_weapon
from utils import add_exceptional_earnings_since_2018

from df_utils import save_csv, get_average_buy_price, get_average_buy_price_df
from datetime import datetime


importlib.reload(lion_weapon)

filepath_list = get_filepaths_list("receipts")
raw_df = Importer().process(filepath_list)
raw_df = add_exceptional_earnings_since_2018(raw_df)


lp = LionWeapon()
df = lp.calculate(raw_df, datetime(2023, 12, 31))
df


  1 of 2 documents processed. (receipts/C6-notaCorretagem-458586-12-07-2024_250125_180652.pdf)
  2 of 2 documents processed. (receipts/C6-notaCorretagem-681604-04-10-2024_250125_180611.pdf)


,abbreviation,preco_medio,qtd
0,BBAS3,26.689940,500.0
1,BBDC4,5.643477,586.0
2,PSSA3,12.372676,2700.0
3,WEGE3,20.324935,1000.0


In [14]:
raw_df[raw_df.abbreviation == "AGRO3"]

,data,compra_venda,abbreviation,titulo,ativo,qtd,preco,valor_operacao,taxas,total_ajustado,preco_ajustado
219,2020-11-10,C,AGRO3,BRASILAGRO,ON,100.0,23.21,2321.0,6.46,2327.46,23.274600
220,2020-11-10,C,AGRO3,BRASILAGRO,ON,300.0,23.21,6963.0,6.46,6969.46,23.231533
221,2020-11-10,C,AGRO3,BRASILAGRO,ON,100.0,23.21,2321.0,6.46,2327.46,23.274600
244,2021-12-28,V,AGRO3,BRASILAGRO,ON,-100.0,27.80,2780.0,4.16,2775.84,27.758400
245,2021-12-28,V,AGRO3,BRASILAGRO,ON,-100.0,27.80,2780.0,4.16,2775.84,27.758400
246,2021-12-28,V,AGRO3,BRASILAGRO,ON,-100.0,27.81,2781.0,4.16,2776.84,27.768400
247,2021-12-28,V,AGRO3,BRASILAGRO,ON,-100.0,27.81,2781.0,4.16,2776.84,27.768400
248,2021-12-28,V,AGRO3,BRASILAGRO,ON,-100.0,27.81,2781.0,4.16,2776.84,27.768400


In [107]:
save_csv(lp.get_raw_df())

# lp.get_position_at_date(datetime(2023, 12, 31))

In [2]:
from df_utils import read, get_sell_df, apply_exceptional_operations, get_profit_df, save_txt
from lion_weapon import format_df

import importlib
import df_utils
importlib.reload(df_utils)

read_filename = 'data.csv'
write_filename = 'stocks_sell_output.txt'

ledger_df = read(read_filename)
ledger_df = format_df(ledger_df)
sell_df = get_sell_df(ledger_df, 2023)
sell_df = apply_exceptional_operations(sell_df)
sell_df = get_profit_df(sell_df)
save_txt(sell_df, write_filename)
sell_df
ledger_df

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [1]:
from df_utils import read, apply_exceptional_operations, get_profit_df, save_txt
from crypto_sell import format_df, get_sell_df

import pandas as pd
import df_utils

import importlib

importlib.reload(df_utils)


read_filename = 'crypto.txt'
write_filename = 'crypto_sell_output.txt'

ledger_df = pd.read_csv(read_filename, sep=";", header=None, names=['data', 'compra_venda', 'qtd', 'abbreviation', 'preco', 'total'])
ledger_df = format_df(ledger_df)
ledger_df['preco'] = ledger_df['preco'].where(ledger_df.preco.notna(), ledger_df['total'] / ledger_df['qtd'])
sell_df = get_sell_df(ledger_df, 2023)
sell_df = get_profit_df(sell_df)
save_txt(sell_df, write_filename)

sell_df


/home/jovyan/work/crypto_sell.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_df.loc[:, "preco_compra"] = sell_df.apply(lambda row: get_average_buy_price(ledger_df, row.abbreviation, row.data), axis=1)


,data,abbreviation,qtd,preco_compra,preco_venda,total_compra,total_venda,lucro_unitario,lucro_total,taxadd
4,2023-11-07,ADA,7152.449700,1.393000,1.665600,9963.362432,11913.120220,0.272600,1949.757788,292.463668
27,2024-11-12,DOGE,12571.693104,0.397592,2.047300,4998.409587,25738.027292,1.649708,20739.617705,3110.942656
28,2024-11-26,XLM,5378.440000,0.693527,2.561623,3730.094497,13777.535469,1.868096,10047.440972,1507.116146


In [65]:
ledger_df.loc[(ledger_df.abbreviation == "PSSA3") & (ledger_df.compra_venda == "C") & (ledger_df.data != datetime(2021, 10, 22)), "preco_ajustado"] /= 3.0  
ledger_df[(ledger_df.abbreviation == "PSSA3") & (ledger_df.compra_venda == "C") 
    ].sort_values("data")

KeyError: 'preco_ajustado'